In [1]:
import pennylane as qml 
import pennylane.numpy as pnp 
import numpy as np
from qas import qml_models 
from qas import mcts
import os 
import json 
from pprint import pprint

dev = qml.device('default.qubit', wires=3)
cwd = os.getcwd() 
print(cwd)

res_file = "20211006-082526.json"
with open(os.path.join(cwd, res_file)) as f:
    res_dict = json.load(f)
print(res_dict.keys())
pprint(res_dict['op_list'])
dev2 = qml.device('default.qubit', wires=7)

d:\Research\QAS-Qiskit\QAS
dict_keys(['task', 'pool', 'params', 'k', 'op_list', 'search_reward_list', 'fine_tune_loss'])
[['CRot', [0, 2], [0.5989389551343641, -1.2288797156243185, 1.317449703785144]],
 ['CNOT', [1, 2], None],
 ['CRot', [1, 2], [1.316438769132651, 0.7164461310611431, 2.1883344760365135]],
 ['CNOT', [1, 2], None],
 ['CRot',
  [0, 1],
  [0.2925405752756747, 0.17417432980808242, -1.045442127572412]]]


In [2]:
@qml.qnode(dev2)
def circ_swap_test_final_circ(params, x=None):

    qml.QubitStateVector(np.kron(x,x), wires = [0,1,2,3,4,5])
    #for i in range(6):
    #    qml.Hadamard(wires=i)

    #qml.Toffoli(wires=[0,1,2])

    
    qml.CRot(params[0], params[1], params[2], wires=[0,2])
    qml.CNOT(wires=[1,2])
    qml.CRot(params[3], params[4], params[5], wires=[1,2])
    qml.CNOT(wires=[1,2])
    qml.CRot(params[6], params[7],params[8], wires=[0,1])
    
    qml.Toffoli(wires=[3,4,5])

    qml.Hadamard(wires=6)
    qml.CSWAP(wires=[6,5,2])
    qml.CSWAP(wires=[6,4,1])
    qml.CSWAP(wires=[6,3,0])
    qml.Hadamard(wires=6)
    return qml.expval(qml.Hermitian(np.outer(qml_models.ket0, qml_models.ket0), wires=6))

def cost_swap_test_final_circ(params):
    x_data = qml_models.TOFFOLI_DATA[0]
    fid = 0
    for i in range(len(x_data)):
        out=circ_swap_test_final_circ(params, x=x_data[i])
        fid = fid + (out-1/2)*2
    return 1-fid/len(x_data)

np.random.seed(0)
var = np.random.randn(9)*0.01

#print(qml.draw(circ_swap_test_full_hadamard)(var))

opt = qml.AdamOptimizer(0.5)
num_epochs = 100
for epoch in range(num_epochs):
    var = opt.step(lambda v: cost_swap_test_final_circ(v), var)
    loss = cost_swap_test_final_circ(var)
    print("Epoch: {}, Loss: {}".format(epoch+1, loss))

Epoch: 1, Loss: 0.28305185605044647
Epoch: 2, Loss: 0.2505583060174257
Epoch: 3, Loss: 0.25489370592069305
Epoch: 4, Loss: 0.2597797579950223
Epoch: 5, Loss: 0.2562674431986426
Epoch: 6, Loss: 0.25253668650247296
Epoch: 7, Loss: 0.2501989633954633
Epoch: 8, Loss: 0.251588702590146
Epoch: 9, Loss: 0.2554246006001666
Epoch: 10, Loss: 0.2555046596438174
Epoch: 11, Loss: 0.2495892743186412
Epoch: 12, Loss: 0.2400652592745307
Epoch: 13, Loss: 0.22329991865937837
Epoch: 14, Loss: 0.2035631336021907
Epoch: 15, Loss: 0.207450437537667
Epoch: 16, Loss: 0.21113721113148687
Epoch: 17, Loss: 0.2026780518960224
Epoch: 18, Loss: 0.2008838013945924
Epoch: 19, Loss: 0.19979664080435822
Epoch: 20, Loss: 0.199112686282649
Epoch: 21, Loss: 0.20022267962926965
Epoch: 22, Loss: 0.19994490322270453
Epoch: 23, Loss: 0.1983007184773664
Epoch: 24, Loss: 0.1958349677343647
Epoch: 25, Loss: 0.19262740412015877
Epoch: 26, Loss: 0.1915699832478348
Epoch: 27, Loss: 0.19440952062055228
Epoch: 28, Loss: 0.19568744526

In [3]:
print(qml.draw(circ_swap_test_final_circ)(var,x=qml_models.TOFFOLI_DATA[0][0]))

 0: ──╭QubitStateVector(M0)──╭C───────────────────────────────────────────────────────────────╭C───────────────────────────────────────╭SWAP─────┤      
 1: ──├QubitStateVector(M0)──│───────────────────────────╭C──╭C───────────────────────────╭C──╰Rot(0.0095, -0.00191, -0.00103)──╭SWAP──│─────────┤      
 2: ──├QubitStateVector(M0)──╰Rot(0.0176, -1.04, -1.55)──╰X──╰Rot(1.59, -1.04, -0.00977)──╰X──╭SWAP─────────────────────────────│──────│─────────┤      
 3: ──├QubitStateVector(M0)──╭C───────────────────────────────────────────────────────────────│─────────────────────────────────│──────├SWAP─────┤      
 4: ──├QubitStateVector(M0)──├C───────────────────────────────────────────────────────────────│─────────────────────────────────├SWAP──│─────────┤      
 5: ──╰QubitStateVector(M0)──╰X───────────────────────────────────────────────────────────────├SWAP─────────────────────────────│──────│─────────┤      
 6: ───H──────────────────────────────────────────────────────────────────────────

In [4]:
# find the best circ throughout search history
search_history = res_dict['search_reward_list']
pool_dict = res_dict['pool']
print(search_history)
print(pool_dict)

[[[8, 7, 4, 7, 6], 0.7479326872984258], [[8, 7, 4, 7, 6], 0.7529638100289385], [[8, 7, 4, 7, 6], 0.7557373653156111], [[8, 7, 4, 7, 6], 0.7571837745295859], [[8, 7, 4, 7, 6], 0.7570566454444388], [[6, 8, 7, 8, 7], 0.7956186929449023], [[4, 7, 3, 8, 4], 0.6851212382790923], [[6, 7, 4, 8, 7], 0.7818023787541579], [[6, 5, 4, 8, 5], 0.7877978777294641], [[4, 3, 7, 8, 4], 0.715107682911261], [[3, 6, 3, 4, 6], 0.7490060119467681], [[8, 6, 7, 6, 7], 0.7501209281110028], [[8, 7, 6, 8, 7], 0.7729716542185956], [[6, 8, 7, 6, 7], 0.8218214215445119], [[8, 4, 3, 8, 3], 0.6683225488729296], [[8, 7, 6, 8, 7], 0.8086102559978708], [[8, 4, 3, 8, 3], 0.6800975015443749], [[8, 7, 6, 8, 7], 0.8380299701064748], [[6, 5, 6, 8, 5], 0.7966056373236011], [[4, 3, 7, 8, 4], 0.7420192027673935], [[4, 3, 7, 8, 4], 0.7453323469630628], [[8, 7, 6, 8, 7], 0.8654622263527878], [[8, 7, 6, 8, 7], 0.8755808484720103], [[8, 7, 6, 8, 7], 0.8880984362141768], [[4, 3, 8, 7, 4], 0.749003330488105], [[4, 3, 4, 7, 4], 0.729635

In [5]:
sorted_search_history=sorted(search_history, key=lambda item: item[1])
print(sorted_search_history)

[[[6, 8, 6, 3, 7], 0.487883240424812], [[6, 8, 4, 5, 7], 0.5742778238378959], [[8, 6, 4, 5, 7], 0.5817369303534529], [[8, 4, 3, 8, 3], 0.6683225488729296], [[8, 4, 3, 8, 3], 0.6800975015443749], [[4, 7, 3, 8, 4], 0.6851212382790923], [[4, 3, 7, 8, 4], 0.715107682911261], [[4, 3, 4, 7, 4], 0.7296354510353106], [[4, 3, 7, 8, 4], 0.7381107315136046], [[8, 7, 4, 7, 6], 0.7382044973661186], [[8, 5, 6, 5, 4], 0.7394064841576989], [[4, 3, 8, 7, 4], 0.7394065684408262], [[4, 3, 7, 8, 4], 0.7403915373687792], [[4, 3, 7, 8, 4], 0.7420192027673935], [[4, 3, 7, 8, 4], 0.7453323469630628], [[8, 7, 4, 7, 6], 0.7479326872984258], [[4, 3, 8, 7, 4], 0.749003330488105], [[3, 6, 3, 4, 6], 0.7490060119467681], [[8, 6, 7, 6, 7], 0.7501209281110028], [[8, 7, 8, 7, 6], 0.7527398991713086], [[8, 7, 4, 7, 6], 0.7529638100289385], [[8, 7, 4, 7, 6], 0.7557373653156111], [[8, 7, 4, 7, 6], 0.7570566454444388], [[8, 7, 4, 7, 6], 0.7571837745295859], [[6, 4, 7, 8, 7], 0.7628241643181652], [[6, 5, 8, 5, 8], 0.7646400

In [6]:
best_k = sorted_search_history[-1][0]
for action in best_k:
    print(pool_dict[str(action)])

{'CRot': [0, 2]}
{'CRot': [1, 2]}
{'CNOT': [0, 2]}
{'CRot': [0, 2]}
{'CNOT': [0, 2]}


In [7]:
@qml.qnode(dev2)
def circ_swap_test_best_historical_circ(params, x=None):

    qml.QubitStateVector(np.kron(x,x), wires = [0,1,2,3,4,5])
    #for i in range(6):
    #    qml.Hadamard(wires=i)

    #qml.Toffoli(wires=[0,1,2])

    
    qml.CRot(params[0], params[1], params[2], wires=[0,2])
    qml.CRot(params[3], params[4], params[5], wires=[1,2])
    qml.CNOT(wires=[0,2])
    qml.CRot(params[6], params[7],params[8], wires=[0,2])
    qml.CNOT(wires=[0,2])
    
    qml.Toffoli(wires=[3,4,5])

    qml.Hadamard(wires=6)
    qml.CSWAP(wires=[6,5,2])
    qml.CSWAP(wires=[6,4,1])
    qml.CSWAP(wires=[6,3,0])
    qml.Hadamard(wires=6)
    return qml.expval(qml.Hermitian(np.outer(qml_models.ket0, qml_models.ket0), wires=6))

def cost_swap_test_best_historical_circ(params):
    x_data = qml_models.TOFFOLI_DATA[0]
    fid = 0
    for i in range(len(x_data)):
        out=circ_swap_test_best_historical_circ(params, x=x_data[i])
        fid = fid + (out-1/2)*2
    return 1-fid/len(x_data)

np.random.seed(0)
var = np.random.randn(9)*0.01

#print(qml.draw(circ_swap_test_full_hadamard)(var))

opt = qml.AdamOptimizer(0.5)
num_epochs = 100
for epoch in range(num_epochs):
    var = opt.step(lambda v: cost_swap_test_best_historical_circ(v), var)
    loss = cost_swap_test_best_historical_circ(var)
    print("Epoch: {}, Loss: {}".format(epoch+1, loss))


Epoch: 1, Loss: 0.22237428022057393
Epoch: 2, Loss: 0.2253553532231729
Epoch: 3, Loss: 0.2044308660213674
Epoch: 4, Loss: 0.20680384909901184
Epoch: 5, Loss: 0.2105422227572833
Epoch: 6, Loss: 0.2057441738738809
Epoch: 7, Loss: 0.19302577993977355
Epoch: 8, Loss: 0.19132762040993168
Epoch: 9, Loss: 0.19751179184641576
Epoch: 10, Loss: 0.1864456467421849
Epoch: 11, Loss: 0.16204134971573203
Epoch: 12, Loss: 0.1371475259451035
Epoch: 13, Loss: 0.10294353812468082
Epoch: 14, Loss: 0.057264997641248705
Epoch: 15, Loss: 0.029133708663433677
Epoch: 16, Loss: 0.0337162007277334
Epoch: 17, Loss: 0.041337816362205904
Epoch: 18, Loss: 0.03551541130353053
Epoch: 19, Loss: 0.029588085996158142
Epoch: 20, Loss: 0.03203302094801963
Epoch: 21, Loss: 0.038244490677362575
Epoch: 22, Loss: 0.04154849202786015
Epoch: 23, Loss: 0.039389852175089723
Epoch: 24, Loss: 0.03228854852876828
Epoch: 25, Loss: 0.021626031659071177
Epoch: 26, Loss: 0.010050388107199382
Epoch: 27, Loss: 0.0021128162049119936
Epoch: 

In [8]:
@qml.qnode(dev)
def circ_expval(params, x=None, y=None):
    qml.QubitStateVector(x, wires = [0,1,2])
    """qml.CRot(params[0], params[1], params[2], wires=[0,2])
    qml.CRot(params[3], params[4], params[5], wires=[1,2])
    qml.CNOT(wires=[0,2])
    qml.CRot(params[6], params[7],params[8], wires=[0,2])
    qml.CNOT(wires=[0,2])"""
    qml.Toffoli(wires=[0,1,2])
    return qml.expval(qml.Hermitian(y, wires=[0,1,2]))

In [9]:
print(qml.draw(circ_swap_test_best_historical_circ)(var,x=qml_models.TOFFOLI_DATA[0][0]))

 0: ──╭QubitStateVector(M0)──╭C─────────────────────────────────────────────────────╭C──╭C───────────────────────────╭C────────────────╭SWAP─────┤      
 1: ──├QubitStateVector(M0)──│──────────────────────────╭C──────────────────────────│───│────────────────────────────│──────────╭SWAP──│─────────┤      
 2: ──├QubitStateVector(M0)──╰Rot(0.0176, 1.58, -1.71)──╰Rot(1.56, -0.00672, 1.61)──╰X──╰Rot(1.36, -1.57, -0.00103)──╰X──╭SWAP──│──────│─────────┤      
 3: ──├QubitStateVector(M0)──╭C──────────────────────────────────────────────────────────────────────────────────────────│──────│──────├SWAP─────┤      
 4: ──├QubitStateVector(M0)──├C──────────────────────────────────────────────────────────────────────────────────────────│──────├SWAP──│─────────┤      
 5: ──╰QubitStateVector(M0)──╰X──────────────────────────────────────────────────────────────────────────────────────────├SWAP──│──────│─────────┤      
 6: ───H──────────────────────────────────────────────────────────────────────────

In [10]:
@qml.qnode(dev)
def circ_toffoli(x=None):
    qml.QubitStateVector(x, wires=[0,1,2])
    qml.Toffoli(wires=[0,1,2])
   
    return qml.state()

@qml.qnode(dev)
def circ_best(params, x=None):
    qml.QubitStateVector(x, wires=[0,1,2])
    
    qml.CRot(params[0], params[1], params[2], wires=[0,2])
    qml.CRot(params[3], params[4], params[5], wires=[1,2])
    qml.CNOT(wires=[0,2])
    qml.CRot(params[6], params[7],params[8], wires=[0,2])
    qml.CNOT(wires=[0,2])
   
    return qml.state()

for i in range(len(qml_models.TOFFOLI_DATA[0])):
    x = qml_models.TOFFOLI_DATA[0][i]
    y = qml_models.TOFFOLI_DATA[1][i]
    print("X:")
    print(x)
    print("Original y:")
    print(y)
    searched_circ_output = circ_best(var, x)
    circ_output_dm = np.matmul(searched_circ_output.reshape((8,1)),np.conj(searched_circ_output.reshape((8,1))).T)
    toffoli_output = circ_toffoli(x)
    print("Searched circ output: ",searched_circ_output)
    print("Toffoli output:       ",toffoli_output)
    print("Toffoli output dm: ")
    print(np.matmul(toffoli_output.reshape((8,1)),np.conj(toffoli_output.reshape((8,1))).T))
    print("Fidelity from swap test: {}".format((circ_swap_test_best_historical_circ(var, x)-1/2)*2))
    print("Fidelity: ", np.dot(np.conjugate(searched_circ_output).T, toffoli_output)*np.conj(np.dot(np.conjugate(searched_circ_output).T, toffoli_output)))
    fidelity_from_dm = np.matmul(np.matmul(np.conj(toffoli_output.reshape((8,1))).T, circ_output_dm), toffoli_output.reshape((8,1)))
    print("Fidelity from Expval (Hand Calculate with pennylane's toffoli output density matrix)",fidelity_from_dm)
    print("Fidelity from expval (with pennylane and target from dataset):",circ_expval(var, x=x, y=y))

X:
[1 0 0 0 0 0 0 0]
Original y:
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
Searched circ output:  [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
Toffoli output:        [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
Toffoli output dm: 
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0